## 用于手写数字识别的简单神经网络（基于MINIST数据集）

### 环境准备

In [29]:
pip install numpy

In [30]:
#### Libraries
# Standard library
import random
import time
import pickle
import gzip

# Third-party libraries
import numpy as np

### 数据定义
- sizes是一个列表（List），通常用于表示神经网络中各层的神经元数量
  - 784：输入层的神经元数量。对于 MNIST 数据集，每个图像是 28x28 像素，展开成一个一维的 784 元素向量，因此输入层有 784 个神经元
  - 30：隐藏层的神经元数量。这里是一个单独的隐藏层，包含 30 个神经元，通常可以根据具体的任务来调整这个数量
  - 10：输出层的神经元数量。对于 MNIST 数据集，输出是一个 10 类分类任务（数字 0 到 9），所以输出层有 10 个神经元，每个神经元表示一个类别的概率
- num_layers是神经网络中层的数量，数值为3，表示这个神经网络有三层，即输入层、隐藏层和输出层
- biases 随机初始化隐藏层和输出层的偏置
  - 第一个元素是一个(30, 1)的数组(30行1列的二维数组)，表示隐藏层的 30 个神经元的偏置
  - 第二个元素是一个(10, 1)的数组，表示输出层的10个神经元的偏置
- weights 随机初始化权重矩阵
  - `zip(sizes[:-1], sizes[1:])` 会得到 [(784, 30), (30, 10)]
  - 输入层到隐藏层的权重矩阵形状是 (30, 784)，隐藏层到输出层的权重矩阵形状是 (10, 30)

In [31]:
sizes = [784, 30, 10]
num_layers = len(sizes)
biases = [np.random.randn(y, 1) for y in sizes[1:]]
weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

### 加载数据集
- 加载一个压缩的 mnist.pkl.gz 文件，其中包含了MINIST数据集，'rb'表示以二进制模式打开该文件
- MNIST数据集在保存时使用了'latin1'编码，需指定解包器使用该编码
- u.load()解包数据。这些数据将用于训练、验证和测试神经网络模型。

In [32]:
f = gzip.open('mnist.pkl.gz', 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
tr_d, va_d, te_d = u.load()
f.close()

### 格式化数据
- tr_d[0]是一个列表，每个元素是一个784维的向量(表示一个28像素*28像素的图像)，我们将其转换为784行1列的列向量
- tr_d[1]是一个列表，每个元素是一个数字(0-9)，称为标签，表示图像对应的数字，我们将其转换为一个 10 维的单位向量
- 将training_inputs和training_results一一对应拼接起来形成格式化的训练数据集traning_data
- 对验证集和测试机做相同的操作，不过数据的标签不需要向量化

In [33]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
training_results = [vectorized_result(y) for y in tr_d[1]]
training_data = list(zip(training_inputs, training_results))
validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
validation_data = list(zip(validation_inputs, va_d[1]))
test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
test_data = list(zip(test_inputs, te_d[1]))

### 训练数据定义
你可以尝试修改这些参数，体会其对训练结果的影响

In [34]:
n_traning = len(training_data) # 训练集的图片数量
n_test = len(test_data) # 测试集的图片数量
epochs = 30 # 训练的轮数
mini_batch_size = 10 # 每个小批量的大小
eta = 1.0 # 学习率

### 训练模型函数 (随机梯度下降)

#### 工具函数
**sigmod函数**

在神经网络里，Sigmoid 常用作神经元的激活函数，把任意输入的实数z压缩到 (0,1) 之间
$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

**sigmod函数的导数**

Sigmoid 函数在任何点都是可导的，而且导数有漂亮的形式（推导简单，方便反向传播训练）
$$
\sigma'(z) = \sigma(z) \times (1 - \sigma(z))
$$

In [35]:
def cost_derivative(output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

#### 反向传播函数
**第一层（输入层到第一隐藏层）特例**：

$$
z^{(1)} = W^{(1)} \cdot x + b^{(1)}
$$

其中 \( x \) 是输入向量。

**每一层的输出（以第 \( l \) 层为例）**：

$$
z^{(l)} = W^{(l)} \cdot \sigma\left(z^{(l-1)}\right) + b^{(l)}
$$

**损失函数**:
$$
C = \frac{1}{2} \| a - y \|^2 \\
C' = a - y
$$

**计算输出层的误差**：
$$
\delta^{(L)} = \nabla_a C \odot \sigma'\left(z^{(L)}\right) \\
= (\sigma\left(z^{(L)}\right) - y) \times \sigma'\left(z^{(L)}\right)
$$
- $\odot$ 在数学里表示的是 Hadamard积，两个同形状的向量或矩阵，对应位置的元素分别相乘。这里输出和标准输出均为0维向量（单个数字），可以相乘。

**输出层的梯度**:

梯度告诉我们如何调整每个偏置和权重，使得损失函数减少。
- 偏置的梯度即为误差
- 权重的梯度为误差点乘前一层激活值的转置。可以理解为第 L 层的权重梯度是当前层的误差项与前一层激活值的外积
$$
\nabla_b^{(L)} = \delta^{(L)}
$$

$$
\nabla_w^{(L)} = \delta^{(L)} \cdot \left(a^{(L-1)}\right)^T
$$

**向后传播误差**:
$$
\delta^{(l)} = \left(W^{(l+1)}\right)^T \delta^{(l+1)} \odot \sigma'\left(z^{(l)}\right)
$$

$$
\nabla_b^{(l)} = \delta^{(l)}
$$

$$
\nabla_w^{(l)} = \delta^{(l)} \cdot \left(a^{(l-1)}\right)^T
$$




In [36]:
# x是每张手写图的向量形式输入，y是标准答案
def backprop(x, y):
   # 初始化偏置和权重的梯度
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]

    # 向前传播
    zs = [] # 存储每一层的输出向量
    activation = x
    activations = [x] # 存储每一层输出向量的激活值（归一化）
    for b, w in zip(biases, weights):
        z = np.dot(w, activation)+b # 计算每一层的输出向量
        zs.append(z)
        activation = sigmoid(z) # 计算每一层输出向量的激活值
        activations.append(activation)

    # 反向传播
    delta = cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) # 计算输出层与标准答案的误差
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())

    # 反向传播误差到每一层
    for l in range(2, num_layers):
        delta = np.dot(weights[-l+1].transpose(), delta) * sigmoid_prime(zs[-l])
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (nabla_b, nabla_w)

### 模型评估函数
所谓评估，就是拿训练好的神经网络去测试训练集

In [37]:
def feedforward(a):
    for b, w in zip(biases, weights):
        a = sigmoid(np.dot(w, a)+b)
    return a

def evaluate(test_data):
    test_results = [(np.argmax(feedforward(x)), y)
                    for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

### 训练并评估
**训练数据预处理**
- 每一轮训练前打乱训练数据的顺序，防止学习过程中模型对数据顺序产生依赖
- 将数据分成一小批一小批的数据（mini-batches），提高效率，也能让模型收敛得更好

In [ ]:
for i in range(epochs):
    time1 = time.time()
    random.shuffle(training_data) # 打乱训练集的顺序
    mini_batches = [training_data[k:k + 10] for k in range(0, n_traning, 10)] # 每个小批量包含10张图片
    mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n_traning, mini_batch_size)]
    for mini_batch in mini_batches:
        nabla_b = [np.zeros(b.shape) for b in biases]
        nabla_w = [np.zeros(w.shape) for w in weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        weights = [w-(eta/len(mini_batch))*nw
                    for w, nw in zip(weights, nabla_w)]
        biases = [b-(eta/len(mini_batch))*nb
                   for b, nb in zip(biases, nabla_b)]
    time2 = time.time()
    if test_data:
        print("Epoch {0}: {1} / {2}, took {3:.2f} seconds".format(i, evaluate(test_data), n_test, time2-time1))
    else:
        print("Epoch {0} complete in {1:.2f} seconds".format(i, time2-time1))

Epoch 0: 7875 / 10000, took 17.01 seconds
Epoch 1: 8206 / 10000, took 7.07 seconds
Epoch 2: 8311 / 10000, took 6.26 seconds
Epoch 3: 9144 / 10000, took 7.88 seconds
Epoch 4: 9255 / 10000, took 5.86 seconds
Epoch 5: 9294 / 10000, took 7.00 seconds
Epoch 6: 9297 / 10000, took 5.92 seconds
Epoch 7: 9319 / 10000, took 6.90 seconds
Epoch 8: 9358 / 10000, took 6.80 seconds
Epoch 9: 9353 / 10000, took 6.55 seconds
Epoch 10: 9378 / 10000, took 6.60 seconds
Epoch 11: 9402 / 10000, took 6.35 seconds
Epoch 12: 9399 / 10000, took 6.96 seconds
Epoch 13: 9401 / 10000, took 6.44 seconds
Epoch 14: 9438 / 10000, took 7.19 seconds
Epoch 15: 9419 / 10000, took 5.94 seconds
Epoch 16: 9413 / 10000, took 7.26 seconds
Epoch 17: 9441 / 10000, took 5.99 seconds
Epoch 18: 9447 / 10000, took 7.87 seconds
Epoch 19: 9444 / 10000, took 5.90 seconds
Epoch 20: 9461 / 10000, took 6.83 seconds
Epoch 21: 9456 / 10000, took 6.16 seconds
Epoch 22: 9458 / 10000, took 6.64 seconds
Epoch 23: 9475 / 10000, took 7.55 seconds
E